In [5]:
import torch
from torch.autograd import Variable
from mpc import mpc
from mpc.mpc import QuadCost, LinDx

In [6]:
torch.manual_seed(0)

n_batch, n_state, n_ctrl, T = 2, 3, 4, 5
n_sc = n_state + n_ctrl

# Randomly initialize a PSD quadratic cost and linear dynamics.
C = torch.randn(T*n_batch, n_sc, n_sc)
C = torch.bmm(C, C.transpose(1, 2)).view(T, n_batch, n_sc, n_sc)
c = torch.randn(T, n_batch, n_sc)

alpha = 0.2
R = (torch.eye(n_state)+alpha*torch.randn(n_state, n_state)).repeat(T, n_batch, 1, 1)
S = torch.randn(T, n_batch, n_state, n_ctrl)
F = torch.cat((R, S), dim=3)

# The initial state.
x_init = torch.randn(n_batch, n_state)

# The upper and lower control bounds.
u_lower = -torch.rand(T, n_batch, n_ctrl)
u_upper = torch.rand(T, n_batch, n_ctrl)

x_lqr, u_lqr, objs_lqr = mpc.MPC(
    n_state=n_state,
    n_ctrl=n_ctrl,
    T=T,
    u_lower=u_lower, 
    u_upper=u_upper,
    lqr_iter=20,
    verbose=1,
    backprop=False,
    exit_unconverged=False,
)(x_init, QuadCost(C, c), LinDx(F))

Initial mean(cost): 3.9041e+01
---------------------------------------------
l              q              r              
---------------------------------------------
iter           0              
mean(cost)     6.680559158325195{:.4e}         
||full_du||_max1.9436144828796387{:.2e}         
mean(alphas)   1.0            {:.2e}         
total_qp_iters tensor([16])   
---------------------------------------------
l              q              r              
---------------------------------------------
iter           1              
mean(cost)     6.441692352294922{:.4e}         
||full_du||_max2.3865065574645996{:.2e}         
mean(alphas)   0.6000000238418579{:.2e}         
total_qp_iters tensor([15])   
---------------------------------------------
l              q              r              
---------------------------------------------
iter           2              
mean(cost)     4.577773094177246{:.4e}         
||full_du||_max0.6194469928741455{:.2e}         
mean(alphas)  